In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter

import imp
import datetime
import LDA_ETM as lda
import scipy
import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import utils as my_utils
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [10]:
dataset_name = "amazon_musical"

In [11]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [12]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,sentiment,summary,unixReviewTime,reviewTime,cleaned,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014","[much, write, doe, exactly, supposed, filter, ...",much write doe exactly supposed filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013","[product, doe, exactly, quite, affordable, rea...",product doe exactly quite affordable realized ...


In [13]:
count_matrix, tfidf_matrix, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [14]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [15]:
barren

array([], dtype=int64)

### Making Edge_Dict

In [8]:
dataset_name

'amazon_musical'

In [9]:
pickle_in = open("resources/edges_amazon_musical_fasttext_nontrained.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [10]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [11]:
len(docs_edges)

10254

In [12]:
edge_dict = []
for doc in docs_edges:
    edge_dict_ = {}
    for i, j in doc:
        try:
            edge_dict_[i] += [j]
        except:
            edge_dict_[i] = [j]
        try:
            edge_dict_[j] += [i]
        except:
            edge_dict_[j] = [i]
    edge_dict.append(edge_dict_)

In [17]:
lambda_param = 1.0
n_top_words = 5

In [7]:
topics_grid = [5, 10, 15, 20, 30, 40, 60, 100]

In [15]:
folder_name = "baseline_etm_amazon_musical_fasttext_nontrained_50iter"

In [16]:
folder_name = str(datetime.datetime.now()) + "_" + folder_name
os.mkdir("dumps/"+folder_name)

In [17]:
import multiprocessing

In [18]:
def multiprocessing_func(k):
    print "Started Sampler...", k
    sampler = lda.LdaSampler(n_topics=k, lambda_param=lambda_param)
    
    sampler.run(count_matrix, edge_dict, maxiter=50)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_n_topics_" + str(k))
    print "Done Sampler...", k
    return k

In [19]:
pool = multiprocessing.Pool()

Started Sampler... 60
Started Sampler... 40
Started Sampler... 30
Started Sampler... 5
Started Sampler... 100
Started Sampler... 20
Started Sampler... 15
Started Sampler... 10


In [ ]:
pool.map(multiprocessing_func, topics_grid)

# Evaluations

In [2]:
folder_name = "2019-09-14 17:16:20.085880_baseline_etm_amazon_musical_fasttext_nontrained_50iter"

In [3]:
# folder_name = "2019-09-14 16:11:52.593802_baseline_etm_amazon_musical_fasttext_nontrained"

In [18]:
def multiprocessing_func_eval(k):
    sampler = joblib.load("dumps/"+folder_name + "/sampler_n_topics_" + str(k))

    t_words = sampler.getTopKWords(n_top_words, words)
    top_words = [t_words[i] for i in t_words.keys()]
    dt_distribution = sampler.theta()
    
#     print ""
#     print "Topics:", k
#     print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
#     print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
#     print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

    print my_utils.coherence_score(count_matrix, top_words, vocabulary), "\t", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)), "\t", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [19]:
# topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [20]:
for k in topics_grid:
    multiprocessing_func_eval(k)

-14.540125146606954 	-0.03961432428768909 	

/home/william18026/miniconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


12.756703862911031
-14.131763524290955 	-0.049593618152886286 	10.423438226007871
-14.705150297680039 	-0.10310896601639405 	10.62825193313079
-15.099200770795168 	-0.08206065236784289 	10.44437956157738
-15.984123197610034 	-0.09433115422003133 	10.150427729950685
-16.542917388278983 	-0.1145657841871095 	9.891144697181995
-17.189850187352327 	-0.12364506388797508 	8.901109606906713
-18.14187090400746 	-0.12338512674400309 	8.37448182532487


# TSNE

In [ ]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()